In [1]:
import pandas as pd
import numpy as np
import pathlib
import math
import tqdm

In [2]:
p = pathlib.Path(r"Z:\Nimrod\experiments\PV_24\27_12_2021\block_12\analysis\re_df.csv")
df = pd.read_csv(p)

In [12]:
# add column with height / width
lst = [df['height']/df['width'] if df['width'] > df['height'] else df['width']/df['height']]
df[r'h_w'] = lst
for
lst = [df.at[i,'height']/df.at[i,'width'] for i in range(len(df['height'])) if df.at[i,'height'] < df.at[i,'width'] else df.at[i,'width']/df.at[i,'height']]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [3]:
# add column with height / width
s = df.height/df.width
s[s>1] = 1/ s[s>1]
df['h_w'] = s

In [4]:
semimajor = df.width
semiminor = df.height
np.sum(semimajor<semiminor)

22012

In [5]:
# Find Eye Center
max_ratio = df.idxmax().h_w
df['h_w'][max_ratio]
aEC = df['center_x'][max_ratio]
bEC = df['center_y'][max_ratio]


In [6]:
# Find f/z0
f_z = math.sqrt( math.pow(df.center_x - aEC, 2) + math.pow(df.center_y - bEC, 2) ) / math.sqrt(1 - math.pow(df.h_w, 2))
df['f_z'] = f_z


TypeError: cannot convert the series to <class 'float'>

In [7]:
for row in tqdm.tqdm(range(len(df))):
    frame = df.iloc[row]['R_eye_frame']
    if frame == frame:
        comp = math.sqrt(math.pow(df.iloc[row]['center_x'] - aEC, 2) + math.pow(df.iloc[row]['center_y'] - bEC, 2)) / math.sqrt((1 - math.pow(df.iloc[row]['h_w'], 2)) )
        df.loc[row, 'f_z'] = comp
    else:
        df.loc[row, 'f_z'] = np.nan

100%|██████████| 106709/106709 [01:03<00:00, 1686.78it/s]


In [32]:
f_z = df['f_z'].median()
for row in tqdm.tqdm(range(len(df))):
    frame = df.iloc[row]['R_eye_frame']
    if frame == frame:
        comp = 2*np.max([df.loc[row,'height'],df.loc[row,'width']]) / f_z
        df.loc[row, 'r'] = comp
    else:
        df.loc[row, 'r'] = np.nan

100%|██████████| 106709/106709 [01:41<00:00, 1047.89it/s]


In [33]:
f_z = df['f_z'].median()
for row in tqdm.tqdm(range(len(df))):
    frame = df.iloc[row]['R_eye_frame']
    if frame == frame:
        comp_t = np.arcsin( (df.loc[row, 'center_x'] - aEC) / f_z)
        comp_p = np.arcsin( (df.loc[row, 'center_y'] - bEC) / (np.cos(comp_t)*f_z) )

        df.loc[row, 'teta'] = comp_t*180/np.pi
        df.loc[row, 'phi2'] = comp_p*180/np.pi
    else:
        df.loc[row, 'teta'] = np.nan
        df.loc[row, 'phi2'] = np.nan

100%|██████████| 106709/106709 [03:05<00:00, 575.37it/s]


In [34]:
df

,Unnamed: 0,Arena_TTL,R_eye_frame,L_values,R_values,center_x,center_y,width,height,phi,ellipse_size,ms_axis,h_w,f_z,r,teta,phi2
0,0,1270728.0,NaN,0.093588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
1,1,1271070.0,3.0,0.060870,-1.524829,326.442608,176.338977,25.831452,29.352219,-0.213711,2381.988313,17.10,0.880051,84.566632,0.820292,30.602287,15.918978
2,2,1271411.0,4.0,0.043252,-1.540584,326.823720,176.153057,26.416670,28.946191,-0.291574,2402.256404,34.15,0.912613,98.887856,0.808945,30.957435,15.798963
3,3,1271752.0,5.0,0.075203,-1.529697,326.395799,175.928459,25.947246,29.102806,-0.212447,2372.334958,51.20,0.891572,88.203047,0.813322,30.558757,15.515149
4,4,1272093.0,6.0,0.187477,-1.500076,326.593284,175.532772,26.306280,29.005546,-0.289193,2397.123190,68.25,0.906940,94.868463,0.810604,30.742541,15.163183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106704,106704,37658090.0,109362.0,-0.376890,-0.200759,NaN,NaN,NaN,NaN,NaN,NaN,1819368.10,NaN,NaN,NaN,NaN,NaN
106705,106705,37658431.0,109363.0,-0.273060,-0.205715,NaN,NaN,NaN,NaN,NaN,NaN,1819385.15,NaN,NaN,NaN,NaN,NaN
106706,106706,37658772.0,109364.0,NaN,-0.190146,NaN,NaN,NaN,NaN,NaN,NaN,1819402.20,NaN,NaN,NaN,NaN,NaN
106707,106707,37659113.0,109365.0,NaN,-0.199242,NaN,NaN,NaN,NaN,NaN,NaN,1819419.25,NaN,NaN,NaN,NaN,NaN


In [12]:
df.to_csv("after_calc.csv")

In [36]:
max_ratio

11856

In [35]:
df.describe()

,Unnamed: 0,Arena_TTL,R_eye_frame,L_values,R_values,center_x,center_y,width,height,phi,ellipse_size,ms_axis,h_w,f_z,r,teta,phi2
count,106709.000000,1.067090e+05,106707.000000,106706.000000,106707.000000,89243.000000,89243.000000,89243.000000,89243.000000,89243.000000,89243.000000,1.067090e+05,89243.000000,89243.000000,89243.000000,89243.000000,89243.000000
mean,53354.000000,1.946805e+07,54692.574320,0.000976,0.000802,308.808944,143.011359,26.249921,24.478026,0.161647,2020.080911,9.098662e+05,0.894442,69.197204,0.748083,15.454607,-14.468746
std,30804.379275,1.050469e+07,31570.527148,0.972381,0.978804,13.039591,15.600163,1.464962,2.324049,0.495205,229.568958,5.252344e+05,0.053620,20.060381,0.032773,10.808060,13.711648
min,0.000000,1.270728e+06,3.000000,-40.777567,-41.853195,260.637622,107.611652,13.222510,9.587387,-0.785386,413.848239,0.000000e+00,0.508348,0.000000,0.369523,-24.232118,-49.542104
25%,26677.000000,1.037082e+07,27352.000000,-0.257794,-0.103526,304.107275,132.600460,25.461839,23.105994,-0.332121,1900.066044,4.550045e+05,0.863513,61.397910,0.729045,11.360320,-23.402852
50%,53354.000000,1.947042e+07,54700.000000,-0.083970,-0.028591,310.838824,140.291198,26.492331,24.253125,0.343810,2024.823944,9.099847e+05,0.898420,71.565272,0.751477,16.920234,-16.803646
75%,80031.000000,2.856546e+07,82033.500000,0.134838,0.115728,318.614379,153.986085,27.352347,26.290436,0.546143,2127.101893,1.364737e+06,0.928823,79.939191,0.770506,23.558794,-4.730689
max,106708.000000,3.765945e+07,109365.000000,1.682037,8.344819,343.438959,206.205827,30.162288,30.100291,0.785391,2716.067897,1.819436e+06,0.999620,838.743802,0.842931,48.294142,43.324457


In [15]:
print(f"max: {df.idxmax().R}")
print(f"min: {df.idxmin().R}")
import cv2
import os

max: 30793
min: 20606


In [16]:


# attempt to view specific frame
maxframe = df.idxmax().R
minframe = df.idxmin().R
cap = cv2.VideoCapture(r"Z:\Nimrod\experiments\PV_24\26_12_2021\block_6\eye_videos\LE\PV24_EE6_640x480_60hz_experiment_1_recording_0\PV24_EE6_LE.mp4")

for i in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
    ret, frame = cap.read()
    if i == minframe and ret:
        cv2.imshow("min_r.png", frame)
        cv2.waitKey(5000)
    if i == maxframe and ret:
        cv2.imshow("max_r.png", frame)
        cv2.waitKey(5000)
    if i == max_ratio and ret:
        cv2.imshow("circle",frame)
        cv2.waitKey(5000)
"""

# get total number of frames
totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# check for valid frame number
if maxframe >= 0 & maxframe <= totalFrames:
    # set frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES,maxframe)

while True:
    ret, frame = cap.read()
    cv2.imshow("Video", frame)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
cap.release()"""
cv2.destroyAllWindows()

In [24]:
p = pathlib.Path(r'Z:\Nimrod\experiments\PV_24\27_12_2021\block_12\eye_videos\RE\PV24_EE12_640x480_60hz_experiment_1_recording_0\PV24_EE12.mp4')
cap = cv2.VideoCapture(str(p))
cap.set(1,100)
ret, I = cap.read()
cap.release()
cv2.imshow('frame',I)

In [25]:
cv2.destroyAllWindows()